Data Ingestion & Schema Analysis

In [0]:
df = spark.read.option("header", True).option("inferSchema", True) \
    .csv("/Volumes/workspace/default/nithyashree/traffic_logs.csv")

df.printSchema()
df.show(5, truncate=False)


root
 |-- LogID: string (nullable = true)
 |-- VehicleID: string (nullable = true)
 |-- EntryPoint: string (nullable = true)
 |-- ExitPoint: string (nullable = true)
 |-- EntryTime: timestamp (nullable = true)
 |-- ExitTime: timestamp (nullable = true)
 |-- VehicleType: string (nullable = true)
 |-- SpeedKMH: integer (nullable = true)
 |-- TollPaid: integer (nullable = true)

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime          |ExitTime           |VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|L001 |V001     |GateA     |GateC    |2024-05-01 08:01:00|2024-05-01 08:20:00|Car        |60      |50      |
|L002 |V002     |GateB     |GateC    |2024-05-01 08:10:00|2024-05-01 08:45:00|Truck      |45      |100     |
|L003 |V003     |GateA     |GateD    |2024-05-01 09:00:00|2024-05-01 09:18:0

In [0]:
from pyspark.sql.types import *

traffic_schema = StructType([
    StructField("LogID", StringType(), True),
    StructField("VehicleID", StringType(), True),
    StructField("EntryPoint", StringType(), True),
    StructField("ExitPoint", StringType(), True),
    StructField("EntryTime", TimestampType(), True),
    StructField("ExitTime", TimestampType(), True),
    StructField("VehicleType", StringType(), True),
    StructField("SpeedKMH", IntegerType(), True),
    StructField("TollPaid", IntegerType(), True)
])

df_manual = spark.read.option("header", True).schema(traffic_schema) \
    .csv("/Volumes/workspace/default/nithyashree/traffic_logs.csv")

df_manual.printSchema()
df_manual.show(5, truncate=False)


root
 |-- LogID: string (nullable = true)
 |-- VehicleID: string (nullable = true)
 |-- EntryPoint: string (nullable = true)
 |-- ExitPoint: string (nullable = true)
 |-- EntryTime: timestamp (nullable = true)
 |-- ExitTime: timestamp (nullable = true)
 |-- VehicleType: string (nullable = true)
 |-- SpeedKMH: integer (nullable = true)
 |-- TollPaid: integer (nullable = true)

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime          |ExitTime           |VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|L001 |V001     |GateA     |GateC    |2024-05-01 08:01:00|2024-05-01 08:20:00|Car        |60      |50      |
|L002 |V002     |GateB     |GateC    |2024-05-01 08:10:00|2024-05-01 08:45:00|Truck      |45      |100     |
|L003 |V003     |GateA     |GateD    |2024-05-01 09:00:00|2024-05-01 09:18:0

Derived Column Creation.

In [0]:
from pyspark.sql.functions import col, expr, when

df2 = df_manual.withColumn(
    "TripDurationMinutes",
    (expr("unix_timestamp(ExitTime) - unix_timestamp(EntryTime)") / 60).cast("int")
).withColumn(
    "IsOverspeed",
    when(col("SpeedKMH") > 60, True).otherwise(False)
)

df2.select("LogID", "VehicleID", "TripDurationMinutes", "IsOverspeed", "SpeedKMH").show()


+-----+---------+-------------------+-----------+--------+
|LogID|VehicleID|TripDurationMinutes|IsOverspeed|SpeedKMH|
+-----+---------+-------------------+-----------+--------+
| L001|     V001|                 19|      false|      60|
| L002|     V002|                 35|      false|      45|
| L003|     V003|                 18|      false|      55|
| L004|     V004|                 20|       true|      80|
| L005|     V005|                 35|      false|      40|
+-----+---------+-------------------+-----------+--------+



Vehicle Behavior Aggregations 

In [0]:
from pyspark.sql.functions import avg, sum, count

# 1. Average speed per VehicleType
df2.groupBy("VehicleType").agg(avg("SpeedKMH").alias("AvgSpeed")).show()

# 2. Total toll collected per EntryPoint
df2.groupBy("EntryPoint").agg(sum("TollPaid").alias("TotalToll")).show()

# 3. Most used ExitPoint
df2.groupBy("ExitPoint").agg(count("*").alias("UsageCount")) \
   .orderBy(col("UsageCount").desc()).limit(1).show()

+-----------+--------+
|VehicleType|AvgSpeed|
+-----------+--------+
|        Car|    70.0|
|       Bike|    55.0|
|      Truck|    45.0|
|        Bus|    40.0|
+-----------+--------+

+----------+---------+
|EntryPoint|TotalToll|
+----------+---------+
|     GateA|       80|
|     GateC|       50|
|     GateB|      170|
+----------+---------+

+---------+----------+
|ExitPoint|UsageCount|
+---------+----------+
|    GateD|         2|
+---------+----------+



Window Functions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, lag

# 1. Rank vehicles by speed within each VehicleType
w1 = Window.partitionBy("VehicleType").orderBy(col("SpeedKMH").desc())
df2.withColumn("SpeedRank", rank().over(w1)).select("VehicleID", "VehicleType", "SpeedKMH", "SpeedRank").show()

# 2. Find last exit time per VehicleID using lag
w2 = Window.partitionBy("VehicleID").orderBy("EntryTime")
df2.withColumn("PrevExitTime", lag("ExitTime").over(w2)).select("VehicleID", "EntryTime", "ExitTime", "PrevExitTime").show()


+---------+-----------+--------+---------+
|VehicleID|VehicleType|SpeedKMH|SpeedRank|
+---------+-----------+--------+---------+
|     V003|       Bike|      55|        1|
|     V005|        Bus|      40|        1|
|     V004|        Car|      80|        1|
|     V001|        Car|      60|        2|
|     V002|      Truck|      45|        1|
+---------+-----------+--------+---------+

+---------+-------------------+-------------------+------------+
|VehicleID|          EntryTime|           ExitTime|PrevExitTime|
+---------+-------------------+-------------------+------------+
|     V001|2024-05-01 08:01:00|2024-05-01 08:20:00|        NULL|
|     V002|2024-05-01 08:10:00|2024-05-01 08:45:00|        NULL|
|     V003|2024-05-01 09:00:00|2024-05-01 09:18:00|        NULL|
|     V004|2024-05-01 09:15:00|2024-05-01 09:35:00|        NULL|
|     V005|2024-05-01 10:05:00|2024-05-01 10:40:00|        NULL|
+---------+-------------------+-------------------+------------+



Session Segmentation 

In [0]:
from pyspark.sql.functions import unix_timestamp, round

# Window by VehicleID ordered by EntryTime
w = Window.partitionBy("VehicleID").orderBy("EntryTime")

# Calculate idle time between trips
session_df = df2.withColumn("PrevExitTime", lag("ExitTime").over(w)) \
    .withColumn("IdleTimeMins", 
        round((unix_timestamp("EntryTime") - unix_timestamp("PrevExitTime")) / 60, 2))

session_df.select("VehicleID", "EntryTime", "PrevExitTime", "IdleTimeMins").show()

+---------+-------------------+------------+------------+
|VehicleID|          EntryTime|PrevExitTime|IdleTimeMins|
+---------+-------------------+------------+------------+
|     V001|2024-05-01 08:01:00|        NULL|        NULL|
|     V002|2024-05-01 08:10:00|        NULL|        NULL|
|     V003|2024-05-01 09:00:00|        NULL|        NULL|
|     V004|2024-05-01 09:15:00|        NULL|        NULL|
|     V005|2024-05-01 10:05:00|        NULL|        NULL|
+---------+-------------------+------------+------------+



Anomaly Detection 

In [0]:
from pyspark.sql.functions import col

# 1. Vehicles with speed > 70 and trip duration < 10 minutes
anomaly_fast_short = df2.filter((col("SpeedKMH") > 70) & (col("TripDurationMinutes") < 10))
anomaly_fast_short.show()

# 2. Vehicles that paid less toll for longer trips (e.g., > 30 mins and toll < 50)
anomaly_low_toll = df2.filter((col("TripDurationMinutes") > 30) & (col("TollPaid") < 50))
anomaly_low_toll.show()

# 3. Suspicious backtracking (ExitPoint alphabetically before EntryPoint)
anomaly_backtrack = df2.filter(col("ExitPoint") < col("EntryPoint"))
anomaly_backtrack.show()


+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime|ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+

+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime|ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+

+-----+---------+----------+---------+

Join with Metadata 

In [0]:
# Load vehicle_registry.csv
vehicle_df = spark.read.option("header", True).csv("/Volumes/workspace/default/nithyashree/vehicle_registry.csv")

# Join on VehicleID
enriched_df = df2.join(vehicle_df, on="VehicleID", how="left")
enriched_df.show()

# Group by RegisteredCity
from pyspark.sql.functions import count

city_summary = enriched_df.groupBy("RegisteredCity").agg(count("*").alias("TotalTrips"))
city_summary.show()

+---------+-----+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+-----------+--------------+
|VehicleID|LogID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|OwnerName|      Model|RegisteredCity|
+---------+-----+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+-----------+--------------+
|     V001| L001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|                 19|      false|     Anil|Hyundai i20|         Delhi|
|     V002| L002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|                 35|      false|   Rakesh| Tata Truck|       Chennai|
|     V003| L003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|  

 Delta Lake Features

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit

# Save DataFrame as Delta table
enriched_df.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/nithyashree/traffic_delta")

# Load DeltaTable
delta_table = DeltaTable.forPath(spark, "/Volumes/workspace/default/nithyashree/traffic_delta")

# Step 1: Update toll for Bikes to 35 using MERGE INTO
bike_updates = enriched_df.filter(col("VehicleType") == "Bike").withColumn("TollPaid", lit(35))

delta_table.alias("target").merge(
    source=bike_updates.alias("source"),
    condition="target.VehicleID = source.VehicleID AND target.EntryTime = source.EntryTime"
).whenMatchedUpdate(set={"TollPaid": col("source.TollPaid")}).execute()

# Step 2: Delete trips longer than 60 mins
delta_table.delete("TripDurationMinutes > 60")

# Step 3: Describe Delta Table History
spark.sql("DESCRIBE HISTORY delta.`/Volumes/workspace/default/nithyashree/traffic_delta`").show()


+-------+--------------------+----------------+--------------------+---------+--------------------+----+--------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|          userId|            userName|operation| operationParameters| job|notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+----------------+--------------------+---------+--------------------+----+--------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      3|2025-06-19 09:40:...|8832284645870584|nithyashreer2019@...| OPTIMIZE|{predicate -> [],...|NULL|    NULL|0619-090812-ipxtt...|          1|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|      2| 2025-06-19 09:40:01|8832284645870584|nithyashreer2019@

 Advanced Conditions

In [0]:
from pyspark.sql.functions import when, col, to_date, count

# 1. Tag trip type as "Short", "Medium", or "Long" based on TripDurationMinutes
enriched_df = enriched_df.withColumn(
    "TripType",
    when(col("TripDurationMinutes") < 15, "Short")
    .when((col("TripDurationMinutes") >= 15) & (col("TripDurationMinutes") <= 30), "Medium")
    .otherwise("Long")
)

# 2. Extract EntryDate for daily trip aggregation
enriched_df = enriched_df.withColumn("EntryDate", to_date("EntryTime"))

# 3. Flag vehicles with more than 3 trips in a day
trip_counts = enriched_df.groupBy("VehicleID", "EntryDate") \
    .agg(count("*").alias("DailyTripCount"))

# Join back to flag frequent vehicles
enriched_df = enriched_df.join(trip_counts, on=["VehicleID", "EntryDate"], how="left") \
    .withColumn("FrequentVehicle", when(col("DailyTripCount") > 3, True).otherwise(False))


Export & Reporting

In [0]:
from pyspark.sql.functions import sum as _sum

# Export 1: Write to Parquet partitioned by VehicleType
enriched_df.write.mode("overwrite").partitionBy("VehicleType") \
    .parquet("/Volumes/workspace/default/nithyashree/traffic_output/parquet")

# Export 2: Write to CSV (for dashboards)
enriched_df.write.mode("overwrite").option("header", True) \
    .csv("/Volumes/workspace/default/nithyashree/traffic_output/csv")

# Export 3: Register SQL view for dashboard summaries
enriched_df.createOrReplaceTempView("traffic_summary")

# Sample View: Total toll by VehicleType and ExitPoint
spark.sql("""
    SELECT VehicleType, ExitPoint, SUM(TollPaid) AS TotalToll
    FROM traffic_summary
    GROUP BY VehicleType, ExitPoint
""").show()


+-----------+---------+---------+
|VehicleType|ExitPoint|TotalToll|
+-----------+---------+---------+
|        Car|    GateC|       50|
|       Bike|    GateD|       30|
|      Truck|    GateC|      100|
|        Car|    GateD|       50|
|        Bus|    GateA|       70|
+-----------+---------+---------+

